# Assignment 2: Filtering

In this assignment we are going to look into filters. Filters are a useful tool to remove noise from a signal, transform a signal or to recover a specific tone/band from a broadband signal, like in a radio.
Here, we will focus on 1D filtering, introduce the signal to noise ration (SNR) and design a very simple FIR filter.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

## Task 1: SNR & Median Filter

In this task, you get a noisy signal sampled (and lowpassed) at different sampling rates.
- Calculate the SNR for the unfiltered signal
- Apply a median filter to the signal
- Calculate the SNR of the filtered signal
- Compare the time domain of both the filtered and unfiltered signal in a plot

In [ ]:
sampling_rates = [2e5, 2e6, 2e7]
samples = 1024*100
tone50k_omega = 2 * np.pi * 50e3

for sampling_rate in sampling_rates:
    time = np.arange(samples) / sampling_rate
    tone50k = np.sin(tone50k_omega * time)
    noisegen = np.random.default_rng()
    noise = 5e-4*noisegen.normal(0.0, 0.5*np.sqrt(sampling_rate), size=samples)

    signal_noisy = tone50k + noise

    # Calculate the SNR in dB as it was introduced in the lecture
    snr = 

    # Calculate the fourier transform of the noisy signal via FFT
    spectrum = 
    f = 

    # Apply a median filter and calculate the new SNR. Assume for simplicity that there is no phase shift!
    # Look here for a function for a median filter:
    # https://docs.scipy.org/doc/scipy/reference/signal.html
    signal_averaged = 
    snr_averaged = 

    print(f"The SNR is {snr} dB at a sampling rate of {sampling_rate}")
    print(f"After filtering, the SNR is {snr_averaged} dB at a sampling rate of {sampling_rate}")

    fig_signal, ax_signal = plt.subplots(figsize=[3.45, 2.3], dpi=200)
    ax_signal.plot(1e6*time, signal_noisy)
    ax_signal.plot(1e6*time, signal_averaged)
    ax_signal.set_xlabel("Time [μs]")
    ax_signal.set_ylabel("Amplitude [a.u.]")
    ax_signal.set_xlim([0,100])
    ax_signal.set_title("Time Domain")

    fig_spectrum, ax_spectrum = plt.subplots(figsize=[3.45, 2.3], dpi=200)
    ax_spectrum.plot(f/1e3, np.abs(spectrum))
    ax_spectrum.set_xlabel("Frequency [kHz]")
    ax_spectrum.set_ylabel("Amplitude [a.u.]")
    #ax_spectrum.set_xlim([0,150])
    ax_spectrum.set_yscale("log")
    ax_spectrum.set_title("Frequency Domain")
    
plt.show()

### Questions
- What does a negative (in dB) SNR mean?
  - YOUR ANSWER
- What effect does the increase of the sampling rate have on the SNR?
  - YOUR ANSWER
- Why is that happening? What is changing with the signal or the noise that causes this? You can relate this to the formula of the SNR.
  - YOUR ANSWER


## Task 2: FIR Filter Design

In this task, the goal is to design a filter that is causal and finite, and thus easy to implement as a digital FIR filter.
This however means that we can not implement an ideal filter!

First, take a look at the following noisy signal.

In [ ]:
sampling_rate = 1e6
samples = 1024*100
tone50k_omega = 2 * np.pi * 50e3
time = np.arange(samples) / sampling_rate
tone50k = np.sin(tone50k_omega * time)
noise = 5e-3*np.random.normal(0, 0.5*np.sqrt(sampling_rate), size=samples)

signal_noisy = tone50k + noise

fig_unfiltered, ax_unfiltered = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_unfiltered.plot(signal_noisy)
ax_unfiltered.set_xlabel("Time [μs]")
ax_unfiltered.set_ylabel("Amplitude [a.u.]")
ax_unfiltered.set_xlim([0,200])
ax_unfiltered.set_title("Unfiltered Signal")

Your task is now to filter this singal to recover the 50 kHz tone from the noisy signal.
- You are going to implement an FIR filter via a windowing method. For this you need to choose two corner frequencies.

Note: You may get two lines in the spectrum, due to taking the absolute of the signal. You can ignore the lower (closer to zero) line.

In [ ]:
# Choose some frequencies for your filters
corners = [x/sampling_rate*2 for x in []] # E.g. [x/sampling_rate*2 for x in [1e6, 2e6]]
exponents = [2, 4, 6, 8]

for exponent in exponents:

    fig_filter, ax_filter = plt.subplots(figsize=[3.45, 2.3], dpi=200)
    fig_spectrum, ax_spectrum = plt.subplots(figsize=[3.45, 2.3], dpi=200)
    fig_output, ax_output = plt.subplots(figsize=[3.45, 2.3], dpi=200)

    taps = 2**exponent

    # Create a FIR filter via the windowing method
    # See: https://docs.scipy.org/doc/scipy/reference/signal.html    
    # Then convolve it with the input to get the filtered signal (output)
    filter = 
    output = 

    # Do a FFT of the filter transfer function
    filter_spectrum = 
    filter_freqs = 

    # Calculate the SNR in dB of the unfiltered and filtered filter. Assume for simplicity that there is no phase shift!
    snr_unfiltered = 
    snr_filtered = 
    
    print(f"Unfiltered SNR {snr_unfiltered} dB with {taps} taps")
    print(f"Filtered SNR {snr_filtered} dB {taps} taps")
    # Phase difference!

    ax_filter.plot(filter)
    ax_spectrum.plot(filter_freqs/1e3, np.abs(filter_spectrum))
    ax_output.plot(1e6*time, signal_noisy, label="noisy")
    ax_output.plot(1e6*time, output, label="filtered")

    ax_filter.set_xlabel("Time [μs]")
    ax_filter.set_ylabel("Amplitude [a.u.]")
    ax_filter.set_title(f"FIR Filter {taps} taps")
    ax_spectrum.set_xlabel("Frequency [kHz]")
    ax_spectrum.set_ylabel("Amplitude [a.u.]")
    ax_spectrum.set_xlim([0,100])
    ax_spectrum.set_ylim([1e-4, 10])
    ax_spectrum.set_yscale("log")
    ax_spectrum.set_title(f"Transformed FIR Filter {taps} taps")
    ax_output.set_xlabel("Time [μs]")
    ax_output.set_ylabel("Amplitude [a.u.]")
    ax_output.set_xlim([0,200])
    ax_output.set_title(f"Filtered Signal {taps} taps")
    ax_output.legend(fontsize="small")

### Questions:
- What kind of filter do we have designed? Of course it is not the ideal version of that since we are using a finite order.
  - YOUR ANSWER
- Look at the time domain of the filter transfer function, which mathematical function does it remind you of? You have learned about it before, but as a different kind of filter/functionality.
  - YOUR ANSWER
- What kind of transform or modification, compared to the simple formulation H[n], was done to that mathematical function to make it exhibit this kind of filter behaviour?
  - YOUR ANSWER
